# Decentralized Market Project
* Create an evnviornment object similar to CDA that will be able to calculate equilbirum, and efficiency, plot supply and demand and prices from simulation.
* Add the ability to save and load the enviornment from a file.
    * Add weeks to the decentralized market as follows:
    * At the beginning of each week values and costs are initialized but location is determined by the ending location from the previous week.
    * Each period of a week consists of a travel/bargain cycle like in the notebook.
    * Modify the ZID trader to have an affinity for other traders.  You decide how that will work.
        * observe/travel/bargain (harder exercise)


* Simulate your dmarket with 20 weeks, 7 periods per week, 24 bargaining rounds per period.
    * Assume a grid of 10 x 10.
    * Save the data from your simulation
    * Build a PyGame program to playback your simulation.
    * Test what happens with and without ZID affinity.

## Environment

### Supply/Demand: class SpotMarketEnvrionment 

* Supply and Demand

* save graphs to a folder for pygame playback

In [1]:
class SpotMarketEnvironment(object):
    """ A class that makes a market environment consisting of buyers who make
        up the demand curve and sellers who make up the supply curve.  This
        class will also calculate market equilibria and plot the supply and
        demand curves and as an option contract prices."""

    def __init__(self, name="example", num_buyers=2, num_sellers=3):
        """ name (str) = the name of the market
            num_Buyers (int) = the number of buyers in the market
            num_sellers(int) = the number of sellers in the market
            Builds buyers and sellers dictioanry entries
        """
        self.name = name
        self.num_buyers = num_buyers
        self.num_sellers = num_sellers
        self.demand = []
        self.supply = []
        self.buyers = {}
        self.sellers = {}

        # equilibrium calculations made in calc_equilibrium
        self.eq_price_low = None
        self.eq_price_high = None
        self.eq_units = None
        self.eq_max_surplus = None

        for buyer_number in range(self.num_buyers):
            buyer_id = "buyer" + str(buyer_number)
            self.buyers[buyer_id] = []  # Empty list of values for buyer_id

        for seller_number in range(self.num_sellers):
            seller_id = "seller" + str(seller_number)
            self.sellers[seller_id] = []  # Empty list of costs for seller_id

    def show_market(self):
        """Prints market name, number of buyers and number of sellers
        """
        print(f"I am market {self.name} with {self.num_buyers} buyers and " \
              f"{self.num_sellers} sellers")
        print("")

    def show_participants(self):
        """Prints buyers and sellers info
        """
        print("Market Participants")
        print("-------------------")
        print("BUYERS")
        print("------")
        for buyer_number in range(self.num_buyers):
            buyer_id = "buyer" + str(buyer_number)
            print(f"{buyer_id} has values {self.buyers[buyer_id]}")
        print()
        print("SELLERS")
        print("-------")
        for seller_number in range(self.num_sellers):
            seller_id = "seller" + str(seller_number)
            print(f"{seller_id} has costs {self.sellers[seller_id]}")
        print("")

    def add_buyer(self, buyer_number, values):
        """Adds a list of buyer_number's values to the self.buyers dictionary
        """
        buyer_id = "buyer" + str(buyer_number)
        self.buyers[buyer_id] = values

    def get_buyer_values(self, buyer_number):
        """Returns buyer_number's values
        """
        buyer_id = "buyer" + str(buyer_number)
        return self.buyers[buyer_id]

    def get_buyers(self):
        return self.buyers

    def add_seller(self, seller_number, costs):
        """Adds a list of seller_number's costs to the self.seller dictionary
        """
        seller_id = "seller" + str(seller_number)
        self.sellers[seller_id] = costs

    def get_seller_costs(self, seller_number):
        """Returns seller_number_s costs
        """
        seller_id = "seller" + str(seller_number)
        return self.sellers[seller_id]

    def get_sellers(self):
        return self.sellers

    def make_demand(self):
        """ Makes demand list by adding participant values to the demand list
            and sorting the list from high to low.
        """
        self.demand = []
        for buyer_id in self.buyers.keys():
            for value in self.buyers[buyer_id]:
                self.demand.append((buyer_id, value))
        self.demand = sorted(self.demand, key=operator.itemgetter(1), \
                             reverse=True)

    def make_supply(self):
        """ Makes supply list by adding participant costs to the supply list
            and sorting the list from low to high.
        """
        self.supply = []
        for seller_id in self.sellers.keys():
            for cost in self.sellers[seller_id]:
                self.supply.append((seller_id, cost))
        self.supply = sorted(self.supply, key=operator.itemgetter(1), \
                             reverse=False)

    def show_supply_demand(self):
        """Prints supply and demand in a table where each row represnts a
           price from high to low.
        """
        supply_and_demand = self.supply + self.demand
        supply_and_demand = sorted(supply_and_demand, key=operator.itemgetter(1), reverse=True)
        print("Unit    ID       Cost  | Value     ID")
        print("---------------------------------------------------------")
        for index, unit in enumerate(supply_and_demand):
            if unit[0][0] == "b":
                print(f"{index + 1:3}{' ' * 20}| {unit[1]:5}    {unit[0]}")
            if unit[0][0] == "s":
                print(f"{index + 1:3}{' ' * 2}{unit[0]}    {unit[1]:5}  |")
        print("")

    def calc_equilibrium(self):
        """ Calculate Competitive Equilbrium information:
            eq_price_high
            eq_price_low
            eq_units
            max_surplus
        """

        self.max_surplus = 0
        self.eq_units = 0
        last_accepted_value = 0
        last_accepted_cost = 0
        first_rejected_value = 0
        first_rejected_cost = 999999999  # big number > max cost ever

        for buy_unit, sell_unit in zip(self.demand, self.supply):
            buyid, value = buy_unit
            sellid, cost = sell_unit
            if value >= cost:
                self.eq_units += 1
                self.max_surplus += value - cost
                last_accepted_value = value
                last_accepted_cost = cost
            else:
                first_rejected_value = value
                first_rejected_cost = cost
                break
        #  Now caluclate equilibrium price range
        if self.eq_units > 1:
            self.eq_price_high = min(last_accepted_value, first_rejected_cost)
            self.eq_price_low = max(last_accepted_cost, first_rejected_value)
        else:
            print("No Equilibrium")

    def show_equilibrium(self):
        #  Print out market equilibrium numbers
        print()
        print("When market {} is in equilibrium we have:".format(self.name))
        print("equilibrium price    = {} - {}".format(self.eq_price_low, self.eq_price_high))
        print("equilibrium quantity = {}".format(self.eq_units))
        print("maximum surplus      = {}".format(self.max_surplus))
        print()

    def get_equilibrium(self):
        return self.eq_units, self.eq_price_low, self.eq_price_high, self.max_surplus

    def plot_supply_demand(self, name, prices=[]):

        """
        First define supply and demand curves
        """
        # make x-axis arrays for demand_units and supply_units
        dunits = [units for units in range(len(self.demand) + 2)]
        sunits = [units for units in range(len(self.supply) + 1)]
        munits = max(len(dunits), len(sunits))

        # make demand values
        max_value = 0
        for buyerid, value in self.demand:
            if value > max_value:  # find the maximum demand value
                max_value = value
        demand_values = [max_value + 1]  # first element is upper range in graph

        for buyerid, value in self.demand:  # get demand tuples
            demand_values.append(value)  # and pull out second element to get value
        demand_values.append(0)  # pull graph down to x axes

        # make suppl values the same way
        supply_costs = [0]  # note first elemnt is used to create lower range of supply values
        for sellerid, cost in self.supply:  # get supply tupples
            supply_costs.append(cost)  # and pull out second element to get cost

        """
        Set up plot
        """
        plt.figure(figsize=(10, 7.5))  # Set plot dimensions
        ax = plt.subplot(111)
        ax.spines["top"].set_visible(False)
        ax.spines["bottom"].set_visible(True)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(True)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        plt.yticks(fontsize=14)
        plt.xticks(fontsize=14)

        """
        Made a bunch of small changes here
        """
        plt.step(dunits, demand_values, label='Demand')
        plt.step(sunits, supply_costs, label='Supply')

        if len(prices) > 0:
            prices.insert(0, prices[0])  # needed to get line segment for the first price
            punits = [unit for unit in range(len(prices))]
            plt.step(punits, prices, label='Prices')

        ax = plt.gca()
        plt.legend(loc='upper center', frameon=False)
        plt.title('Supply and Demand')
        plt.xlabel('units')
        plt.ylabel('currrency')

        # Save figure in the working directory
        plt.savefig(name+'supply_demand.jpg')

        plt.xlim(0, munits)
        plt.ylim(0, max(demand_values + supply_costs))
        plt.show()



## Instition

### Travel: class Travel

In [2]:
     
class Travel(object):
    """Travel Institution"""
    
    def __init__(self, grid_dimension, agents, debug):
        self.grid_dimension = grid_dimension  # determines dimensions of a square grid  
        self.agents = agents  # list of agent objects
        self.grid = {}  # grid will be a dictionary
        self.history = {}
        self.debug = debug
 
    def start_travel(self):
        self.setup_agents_history()
        self.locate_agents()

    def setup_agents_history(self):
        for agent in self.agents:
            loc = agent.get_location()
            self.history[agent.name] = [loc]
        if self.debug:
            print("starting history", self.history)
         
    def locate_agents(self):
        """Put agents in grid"""
        for agent in self.agents:
            loc = agent.get_location()
            if loc in self.grid.keys():
                self.grid[loc].append(agent) # add agent to list of agents at loc
            else:
                self.grid[loc] = [agent]  # start a list of agents
        
    def get_grid(self):
        return self.grid

    def set_debug(self, debug):
        self.debug = debug 
    
    def run(self):
        for point in self.grid:
            agent_order =[]
            for agent in self.grid[point]:
                agent_order.append(agent)
            rnd.shuffle(agent_order)
            for agent in agent_order:
                msg = Message('MOVE_REQUESTED', 'TRAVEL', agent.get_name(), "  ")
                return_msg = agent.process_message(msg)
                if return_msg.get_directive() == "MOVE":
                    x_dir, y_dir = return_msg.get_payload()
                    loc = agent.get_location()
                    # debug message
                    if self.debug:
                        print(f"Travel -> agent {agent.name} at {loc} moves ({x_dir, y_dir})" +
                              "to ", end = "")
                    if 0 <= loc[0] + x_dir and loc[0] + x_dir <= self.grid_dimension - 1:
                        if 0 <= loc[1] + y_dir and loc[1] + y_dir <= self.grid_dimension - 1:
                            location = loc[0] + x_dir, loc[1] + y_dir
                            if self.debug:
                                print("move good", location)
                            agent.set_location(location)
                            self.history[agent.name].append(location)
                        else:
                            agent.set_location(loc)
                            self.history[agent.name].append(loc)
                            if self.debug:
                                print("move bad", loc)
                    else:
                        agent.set_location(loc)
                        self.history[agent.name].append(loc)
                        if self.debug:
                            print("move bad", loc)
        self.grid = {}
        self.locate_agents()
                
    def print_grid(self):
        for x in range(self.grid_dimension):
            for y in range (self.grid_dimension):
                if (x,y) in self.grid.keys():
                    names = []
                    for agent in self.grid[(x,y)]:
                        names.append(agent.get_name())
                    print(f"at point {(x, y)} we have {len(self.grid[(x,y)])}" + 
                          f" agents = {names}")
    def get_history(self):
        return self.history


### Bargaining: class Bargaining

In [3]:
class Bargain(object):
    def __init__(self, rounds, debug):
        self.agents = []      # list of agent objects
        self.offer_history = []    # list of offer tupples
        self.contracts = []   # list of contract tupples
        self.order_book = {}  # dictionary key=trader_id, value = (type, amount)
        self.agent_order = []  # list of shuffled agents
        self.agent_lookup = {} # dictionary key=trader_id, value = index into agent_order
        self.rounds = rounds  # number of rounds of bargaining
        self.debug = debug  # used to print verrbose information for debugging
        
    def set_debug(self, flag):
        self.debug = flag
    
    def send_msg(self, agent, msg):
        if self.debug:
            directive = msg.get_directive()
            sender = msg.get_sender()
            receiver = msg.get_receiver()
            payload = msg.get_payload()
            print(f"message sent = {directive} from {sender} to {receiver}, \n {10*' '}{payload}")
        return_msg = agent.process_message(msg)
        return return_msg
        
    def run(self):
        """Runs bargaining in rounds at each point in the grid"""
        
        self.agent_order = self.agents
        self.order_book = {}
        for k, agent in enumerate(self.agent_order):
            name = agent.get_name()
            self.order_book[name] = None
            self.agent_lookup[name] = k  # See comments in "BUY and SELL code below"
            
        #self.offer_history = [] # tuples of form (round, trader_id, type, amount)
        #self.contracts = [] # tuples of the form (round, price, buyer_id, seller_id)
        
        for round in range(self.rounds):
            rnd.shuffle(self.agent_order)
            for k, agent in enumerate(self.agent_order):
                name = agent.get_name()
                self.agent_lookup[name] = k  # See comments in "BUY and SELL code below"
            for agent in self.agent_order:
                agent_id = agent.get_name()
                msg = Message('ACTION_REQUESTED', 'BARGAIN', agent_id, self.order_book)
                return_msg = self.send_msg(agent, msg)
                directive = return_msg.get_directive()
                sender_id = return_msg.get_sender()
                payload = return_msg.get_payload()
                if directive == "NULL":
                    continue
                if directive == "BID":
                    offer = ("BID", payload)
                    self.order_book[sender_id] = offer
                    self.offer_history.append((round, sender_id, "BID", payload)) 
                elif return_msg.get_directive() == "ASK":
                    offer = ("ASK", payload)
                    self.order_book[sender_id] = offer
                    self.offer_history.append((round, sender_id, "ASK", payload)) 
                elif directive == "BUY":
                    buyer_id = sender_id  # Get Mappings to buyer_id and seller_id
                    seller_id = payload
                    if self.order_book[seller_id] == None:
                        continue
                    # process contract
                    price = self.order_book[seller_id][1]
                    self.offer_history.append((round, buyer_id, "BUY", price))
                    contract = (round, price, buyer_id, seller_id)
                    self.contracts.append(contract)
                    # valid contract cancels previous orders
                    # note in asynchronous case could put in hold and decide what to do
                    # based on return message
                    self.order_book[buyer_id] = None
                    self.order_book[seller_id] = None
                    # Send messages to buyer and seller that they have a contract
                    msg = Message('CONTRACT', 'BARGAIN', buyer_id, contract) 
                    return_msg = agent.process_message(msg) # Send to buyer
                    msg = Message('CONTRACT', 'BARGAIN', seller_id, contract)
                    # return_msg = ?.process_message(msg) # Send to seller
                    # this is a little complicated.  I need a reverse dictionary from
                    # trader_name to agent object.  I will simply build one for now
                    # and later refactor to shuffle list of trader names and not onjects
                    seller_agent_loc = self.agent_lookup[seller_id]
                    seller_agent = self.agent_order[seller_agent_loc] 
                    return_msg = seller_agent.process_message(msg)  # Send to seller
                elif directive == "SELL":
                    seller_id = sender_id  # Get Mappings to buyer_id and seller_id
                    buyer_id = payload
                    if self.order_book[buyer_id] == None:
                        continue
                    # process contract
                    price = self.order_book[buyer_id][1]
                    self.offer_history.append((round, seller_id, "SELL", price))
                    contract = (round, price, buyer_id, seller_id)
                    self.contracts.append(contract)
                    # valid contract cancels previous orders
                    # note in asynchronous case could put in hold and decide what to do
                    # based on return message
                    self.order_book[buyer_id] = None
                    self.order_book[seller_id] = None
                    # Send messages to buyer and seller that they have a contract
                    msg = Message('CONTRACT', 'BARGAIN', seller_id, contract) 
                    return_msg = agent.process_message(msg) # Send to buyer
                    msg = Message('CONTRACT', 'BARGAIN', buyer_id, contract)
                    # return_msg = ?.process_message(msg)
                    # this is a little complicated.  I need a reverse dictionary from
                    # trader_name to agent object.  I will simply build one for now
                    # and later refactor to shuffle list of trader names and not onjects
                    buyer_agent_loc = self.agent_lookup[buyer_id]
                    buyer_agent = self.agent_order[buyer_agent_loc] 
                    return_msg = buyer_agent.process_message(msg)  # Send to seller
                else:
                    return Message('BAD', agent.get_name(), 'BARGAIN',
                                   "Unrecognized Directive") 
        
    
    def set_agents(self, agents):
        self.agents = agents
    
    def get_offer_history(self):
        return self.offer_history()
    
    def get_contracts(self):
        return self.contracts
    
    def get_prices(self, typ, name):
        """Get all contract prices in order for a trader"""
        prices = []
        for contract in self.contracts:
            round_t, price, buyer_id, seller_id = contract
            if typ == "BUYER":
                if name == buyer_id:
                    prices.append(price)
            elif typ == "SELLER":
                if name == seller_id:
                    prices.append(price)
        return prices

    def print_payoffs(self, loc):
        print(f"PAYOFFS for location {loc}")
        print("-----------------------------")
        for agent in self.agents:
            if agent.type == "BUYER":
                prices = self.get_prices(agent.type, agent.name)
                utility = agent.get_payoff(prices)
                print(f"{agent.name} has utility {utility}")
        print()
        for agent in self.agents:
            if agent.type == "SELLER":
                prices = self.get_prices(agent.type, agent.name)
                profit = agent.get_payoff(prices)
                print(f"{agent.name} has profit {profit}")



### Trader Action: class Trader

In [4]:
# changes in class Trader:
class Trader(object):
    """Base class for Buyers or Seller Agents
       Decision making is provided by a child class where 
       overridden methods are those called in process_message
    """
    def __init__(self, name, trader_type, payoff, money, location,
             lower_bound = 100, upper_bound = 500):
        
        self.simulation = None     # get access to class SimulateMarket
        
    def get_simulation(self, simulation):
        self.simulation = simulation
        return self.simulation
    
    
    

# In class SimulateMarket
class SimulateMarket(object):
    """Simulate a market period using two types of trading agents"""
    
    def make_whole_trader_list(self):
        """
        build self.traders for each week: name, type, payoff, money, initial location
        location will be inherited from last day
        generate new res_values
        """
        if self.current_week == 0:
            self.build_traders()
            self.trader_dic[self.current_week] = self.trader_list
        else:
            trader_list = []
            for i in range(self.num_traders):
                t = self.trader_dic[self.current_week-1][i]
                agent_model = type(t)
                agent = agent_model(t.name, t.type, t.payoff, t.money, t.location)   # inherit location
                trader_list.append(agent)
            self.trader_dic[self.current_week] = trader_list
        for i in range(self.num_traders):
            t = self.trader_dic[self.current_week][i]                 # get single trader class instance
            values = self.gen_res_values(True, mu=400, sigma=200)     # generate values if traders is buyer
            t.set_values(values)
            if i >= self.num_traders//2:
                costs = sorted(values, reverse=False)                 # Convert values to costs
                t.set_costs(costs)
            t.get_simulation(self)                                    # trader can get access to simulation results.


### Procedure: class SimulateMarket

In [5]:
class SimulateMarket(object):
    """Simulate a market period using two types of trading agents"""

    def __init__(self, sim_name, num_weeks, num_periods, num_rounds, num_traders, trader_types, grid_size, num_units , debug):

        self.sim_name = sim_name             # simulation name
        self.num_weeks = num_weeks                                     # simulation weeks
        self.num_periods = num_periods       # simulation periods(days) for a week
        self.num_rounds = num_rounds         # number of bargaining rounds for a day
        self.trader_types = trader_types     # list of two trader types  SHOULD BE TOPPLE
        self.grid_size= grid_size                                      # simulation grid size: square
        self.num_traders = num_traders       # number of traders divisible by two
        self.num_units = num_units  # number of units, same for all traders
        self.debug = debug

        self.current_week = 0                                          # initial week = 0
        self.location_list = []                                        # initial location list for all traders
        self.trader_dic = {}                                           # trader list for all weeks
        self.results_whole = {}              # whole simulation results(moving history, market conditions), key = week

        self.auction = None  # double auction object
        self.auction_limits = (999, 0)  # tuple of limits, (max bid, min ask)
        self.market = None  # market environment object
        self.traders = None  # list of participating trader class names
        self.trader_list = None  # list of initialized trader objects
        self.prices = {}                                               # dic of prices from contracts, key = week
        self.contracts = {}                                            # dic of full contracts, key = week

        self.buyer_surplus = None  # Surplus generate by buyers. Sum of (value-price)
        self.seller_surplus = None  # Surplus generated by sellers.  Sum of (price-cost)
        self.actual_surplus = None  # Sum of buyer surplus and seller surplus
        self.efficiency = None  # (actual_surplus/eq_max_surplus) * 100.
        self.type_surplus = {}
        self.sim_time = [0.0]  # sim_time[0] = round; sim_time[1] = tick

    def gen_location_list(self):
        """
        Initialize the location list
        Returns a sorted list of location (x, y) drawn from Uniform distribution truncated to be non-negative
        """
        for i in range(self.num_traders):
            x = rnd.randint(0,self.grid_size)
            y = rnd.randint(0,self.grid_size)
            self.location_list.append((x, y))

    def build_traders(self):
        """
        build self.traders: name, type,payoff, money, initial location
        """
        # replicate trade_object total_traders//2 times and put in traders list
        # make a shuffled list of trader objects for trader roles
        self.gen_location_list()

        self.traders = []
        for k in range(self.num_traders // 2):  # // produces integer result
            self.traders = self.traders + self.trader_types
        np.random.shuffle(self.traders)

        # Assign trader objects to buyer/seller roles and assign values and costs

        self.trader_list = []
        for t in range(self.num_traders):
            agent = None
            name = "B_{}".format(t + 1)
            trader_type = "BUYER"
            payoff = utility
            money = 500
            if t >= self.num_traders // 2:
                name = "S_{}".format(t + 1 - self.num_traders // 2)
                trader_type = "SELLER"
                payoff = profit
            agent_model = self.traders[t]
            agent_kind = str(agent_model.__name__)
            name = name + "_{}".format(agent_kind)
            location = self.location_list[t]                                  # get initial location
            agent = agent_model(name, trader_type, payoff, money, location)
            self.trader_list.append(agent)

    def make_whole_trader_list(self):
        """
        build self.traders for each week: name, type, money, initial location
        location will be inherited from last day
        generate new res_values
        """
        if self.current_week == 0:
            self.build_traders()
            self.trader_dic[self.current_week] = self.trader_list
        else:
            trader_list = []
            for i in range(self.num_traders):
                t = self.trader_dic[self.current_week-1][i]
                agent_model = type(t)
                agent = agent_model(t.name, t.type, t.payoff, t.money, t.location)   # inherit location
                trader_list.append(agent)
            self.trader_dic[self.current_week] = trader_list
        for i in range(self.num_traders):
            t = self.trader_dic[self.current_week][i]                 # get single trader class instance
            values = self.gen_res_values(True, mu=400, sigma=200)     # generate values if traders is buyer
            t.set_values(values)
            if i >= self.num_traders//2:
                costs = sorted(values, reverse=False)                 # Convert values to costs
                t.set_costs(costs)
            t.get_simulation(self)                                    # trader can get access to simulation results.


    def run_simulation(self):
        """ Runs a simulation for a week"""
        debug = self.debug
        agent_list = self.trader_dic[self.current_week]
        t_inst = Travel(self.grid_size, agent_list, debug)
        t_inst.start_travel()
        b_inst = Bargain(self.num_rounds, debug)
        print(f"{15*'='} starting grid")
        t_inst.print_grid()
        
        self.contracts[self.current_week] = {}
        self.prices[self.current_week] = []
        
        # show Monday-Sunday in results
        period_ls = ["New Start: Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        for period in range(self.num_periods):
            self.contracts[self.current_week][period_ls[period]] = []              #accumulated contracts for every day
            print(f"{40 * '='}  {period_ls[period]}")
            t_inst.run()
            if debug:
                print(f"{20*'='} grid")
                t_inst.print_grid()
            g = t_inst.get_grid()
            for loc in g:
                agents = g[loc]
                # Run bargain if you have a BUYER and A Seller
                buyer_found = False
                seller_found = False
                for agent in agents:
                    if agent.type == "BUYER":
                        buyer_found = True
                    if agent.type == "SELLER":
                        seller_found = True
                if buyer_found and seller_found:
                        b_inst.set_agents(agents)
                        b_inst.set_debug(self.debug)
                        b_inst.run()
            self.contracts[self.current_week][period_ls[period]].extend(b_inst.get_contracts())
            print_contracts(self.contracts[self.current_week][period_ls[period]])

        print(f"{20*'-'}")
        if debug:
            for agent in agent_list:
                print(f"{agent.name} transacted {agent.get_units_transacted()}")

        # store moving history to self.result_whole
        self.results_whole[self.current_week] = {}
        history_of_travel = t_inst.get_history()
        self.results_whole[self.current_week]["Moving_History"] = history_of_travel
        for agent_key in history_of_travel:
            print(f"{agent_key} moved {history_of_travel[agent_key]}")

        # store prices list to self.prices(dict)
        contracts = b_inst.get_contracts()
        # self.contracts[self.current_week].extend(contracts)
        for i in range(len(contracts)):
            self.prices[self.current_week].append(contracts[i][1])




    def make_whole_simulation(self):
        """
        make a whole session: multi-week
        """
        for week in range(self.num_weeks):
            print(f"{28 * '='} Week {week}")
            self.make_whole_trader_list()           # build trader for every week
            self.make_market()                      # set up market environment for plot
            self.run_simulation()                   # run simulation for every week
            self.plot_prices()                      # plot supply_demand_prices, and save in a folder
            self.display_results()                  # display results
            self.get_results()                      # store other parameters(num_trader...) to results_whole
            if self.debug:
                print(self.get_results())
            self.current_week = self.current_week + 1

        # save the dict results_whole to a json file, in the same folder where graphs are
        json.dump(self.results_whole, open(str(self.sim_name) + "/Results_json.json", "w"))

        
        
    def plot_prices(self):
        print(self.prices[self.current_week])
        # save graph to the folder with specific file name
        self.market.plot_supply_demand(name= str(self.sim_name)+"/Week"+str(self.current_week), prices=self.prices[self.current_week])

        
    def get_results(self):
        """for one week: return results as dictionary"""

        total_price = 0.0
        if len(self.prices[self.current_week]) == 0:
            avg_price = 0
        else:
            for price in self.prices[self.current_week]:
                total_price += price
            avg_price = total_price / len(self.prices[self.current_week])

        eq_units, eq_plow, eq_phigh, eq_max_surplus = self.market.get_equilibrium()
        self.calc_efficiency()

        results = self.results_whole[self.current_week]

        results["name"] = self.sim_name
        results['week'] = self.current_week
        results['num_days'] = self.num_periods                   # record num_days for a week
        results['grid_size'] = self.grid_size
        results["number traders"] = self.num_traders
        results["number units"] = self.num_units

        # TODO Add trader types, values, and costs here

        results["eq low price"] = eq_plow
        results["eq high price"] = eq_phigh
        results["eq quantity"] = eq_units
        results["eq surplus"] = eq_max_surplus

        results["avg price"] = avg_price
        results["prices"] = self.prices[self.current_week]
        results["quantity"] = len(self.prices[self.current_week])

        results["buyer surplus"] = self.buyer_surplus
        results["seller surplus"] = self.seller_surplus
        results["actual surplus"] = self.actual_surplus
        results["efficiency"] = self.efficiency
        for trader_strategy in self.type_surplus[self.current_week]:
            results[trader_strategy] = self.type_surplus[self.current_week][trader_strategy]

        return self.results_whole[self.current_week]


    def make_market(self):
        """Make MarketEnviornment object from traders
        """
        # self.build_traders()
        num_side = self.num_traders // 2
        self.market = SpotMarketEnvironment(name="self.sim_name", num_buyers=num_side, num_sellers=num_side)
        for index, trader in enumerate(self.trader_dic[self.current_week]):
            if index < num_side:  # This is a buyer
                values = trader.get_values()
                self.market.add_buyer(index, values)
            else:  # this is a seller
                seller_index = index - num_side  # sellers start at 0 in market environment
                costs = trader.get_costs()
                self.market.add_seller(seller_index, costs)
        self.market.make_demand()
        self.market.make_supply()
        self.market.calc_equilibrium()

    def display_market_equilibrium(self):
        self.market.display_equilibrium()

    def plot_market(self):
        self.market.plot_supply_demand()

    def calc_efficiency(self):
        """
        """

        self.buyer_surplus = 0
        self.seller_surplus = 0
        self.actual_surplus = 0
        self.efficiency = 0

        self.type_surplus[self.current_week] = {}
        for trader in self.trader_list:
            trader_strategy = trader.name.split("_")[-1]  # trader.name = trader_t_type
            trader_surplus = 0
            unit = 0
            # res = trader.get_value_costs()  # get reservation values (values for buyers costs for sellers)
            for contract in list(self.contracts[self.current_week].items())[-1][-1]:
                round_number, price, buyer_name, seller_name = contract
                if trader.type == "BUYER":
                    res = trader.get_values()
                    if trader.name == buyer_name:
                        surplus = res[unit] - price
                        unit = unit + 1
                        trader_surplus = trader_surplus + surplus
                        self.buyer_surplus = self.buyer_surplus + surplus
                        self.type_surplus[self.current_week][trader_strategy] = self.type_surplus[self.current_week].get(trader_strategy,
                                                                                   0) + surplus
                else:
                    res = trader.get_costs()
                    if trader.name == seller_name:
                        surplus = price - res[unit]
                        unit = unit + 1
                        trader_surplus = trader_surplus + surplus
                        self.seller_surplus = self.seller_surplus + surplus
                        self.type_surplus[self.current_week][trader_strategy] = self.type_surplus[self.current_week].get(trader_strategy,
                                                                                   0) + surplus

            self.actual_surplus = self.buyer_surplus + self.seller_surplus
            eq_units, eq_plow, eq_phigh, eq_max_surplus = self.market.get_equilibrium()
            self.efficiency = (self.actual_surplus / eq_max_surplus) * 100.0

    def display_contracts(self):
        """Display a list of contracts"""
        print(f"{'-'*20 } Display Contract")
        for week in range(self.num_weeks):
            print("-"*40 + f"Contracts in Week {week}:")
            for contract in self.contracts[week]:
                round_number, price, buyer_name, seller_name = contract
                print(f"In Round {round_number}, at price {price} Buyer {buyer_name} trades with Seller {seller_name}")

    def display_results(self):
        """Displays a summary of the simulation
        """
        # print("-" * 20 + "Display Results")
        print(f"{40 *'-'} Results in Week {self.current_week}")
        total_price = 0.0
        if len(self.prices[self.current_week]) == 0:
            avg_price = 0
        else:
            for price in self.prices[self.current_week]:
                total_price += price
            avg_price = total_price / len(self.prices[self.current_week])

        eq_units, eq_plow, eq_phigh, eq_max_surplus = self.market.get_equilibrium()
        self.calc_efficiency()

        print(f"Market {self.sim_name} has {self.num_traders} traders, each with {self.num_units} units")
        print(f"      equilibrium price range is {eq_plow} to {eq_phigh}")
        print(f"      average price = {avg_price}")
        print(f"      equilibrium expected contracts = {eq_units}, actual contracts = {len(self.prices[self.current_week])}")
        print(f"      buyer surplus = {self.buyer_surplus}, seller surplus = {self.seller_surplus}")
        print(f"      actual_surplus = {self.actual_surplus}, maximum surplus = {eq_max_surplus}")
        print("       efficiency = {:0.2f}%.\n".format(self.efficiency))
        for trader_strategy in self.type_surplus[self.current_week]:
            print(f"  {trader_strategy} has surplus {self.type_surplus[self.current_week][trader_strategy]}")


    def get_contracts(self):
        return self.auction.get_contracts()

    def gen_res_values(self, buyer_flag, mu=500, sigma=100):
        """Returns a sorted list of values or costs drawn from a Normal distribution truncated to be non-negative
            units = number of draws
            return_values = True (return values), = False (return cost)
            mu = mean of normal distribution
            sigma = standard deviation of normal distribution
        """
        values = []
        unit = 0
        while unit < self.num_units:
            value = round(np.random.randint(sigma, mu))
            if value >= 0:
                values.append(value)
                unit = unit + 1
        if buyer_flag:
            return sorted(values, reverse=True)  # List of values
        else:
            return sorted(values, reverse=False)  # list of costs

    def get_prices(self):
        return self.prices

    def get_traders(self):
        return self.trader_list

    def get_offers(self):
        return self.auction.get_orders()

    def get_buyers(self):
        return self.market.get_buyers()

    def get_sellers(self):
        return self.market.get_sellers()



## Strategies

### Zero Intelligence: Class ZID (Trader)

In [6]:
class ZID(Trader):
    """
        Zero Intelligence variant for decentralized market
        a budget constrained ZI
    """

    def start(self, pl):
        """
        Sets up values for trading
        Does not use payload - pl
        """
        self.units_transacted = 0
        self.cur_unit = 0
        if self.type == "BUYER":
            self.max_units = len(self.values)
        else:
            self.max_units = len(self.costs)
        return_msg = Message("Initial", self.name, self.name, "Initialized")
        self.returned_msg(return_msg)
        return return_msg

    def move_requested(self, pl):
        """
        Make a move in a random direction if you can still trade
        """
        direction_list = [-1, 0, +1]  #
        if self.cur_unit > self.max_units:
            return_msg = Message("MOVE", self.name, "Travel", (0, 0))
            self.returned_msg(return_msg)
            return return_msg
        else:
            x_dir = rnd.choice(direction_list)
            y_dir = rnd.choice(direction_list)
            return_msg = Message("MOVE", self.name, "Travel", (x_dir, y_dir))
            self.returned_msg(return_msg)
            return return_msg

    def action_requested(self, pl):
        """
        Make a bid or ask
        """
        if self.debug:
            print(f"-- {self.name} has {self.units_transacted} of {self.max_units}")
            print(f"-- {self.name} working on unit {self.cur_unit}")
        if self.cur_unit >= self.max_units:
            return_msg = Message("NULL", self.name, "BARGAIN", None)
            self.returned_msg(return_msg)
            return return_msg

        current_offers = pl

        if self.type == "BUYER":
            WTP = rnd.randint(self.lower_bound, self.values[self.cur_unit])
            offers = []
            for trader_id in current_offers:
                if current_offers[trader_id] == None:
                    continue
                offer_type = current_offers[trader_id][0]
                offer_amount = current_offers[trader_id][1]
                if offer_type == "ASK":
                    offers.append((trader_id, offer_amount))
            # Now find an offer
            if len(offers) > 0:
                offer = rnd.choice(offers)
                if WTP >= offer[1]:  # offer[1] = sellers willingness to accept
                    seller_id = offer[0]
                    return_msg = Message("BUY", self.name, "BARGAIN", seller_id)
                    self.returned_msg(return_msg)
                    return return_msg
                else:
                    return_msg = Message("BID", self.name, "BARGAIN", WTP)
                    self.returned_msg(return_msg)
                    return return_msg
            else:
                return_msg = Message("BID", self.name, "BARGAIN", WTP)
                self.returned_msg(return_msg)
                return return_msg

        else:  # for SELLER
            WTA = rnd.randint(self.costs[self.cur_unit], self.upper_bound)
            offers = []
            for trader_id in current_offers:
                if current_offers[trader_id] == None:
                    continue
                offer_type = current_offers[trader_id][0]
                offer_amount = current_offers[trader_id][1]
                if offer_type == "BID":
                    offers.append((trader_id, offer_amount))
            # Now find an offer
            if len(offers) > 0:
                offer = rnd.choice(offers)
                if WTA <= offer[1]:  # offer[1] = buyers willingness to pay
                    buyer_id = offer[0]
                    return_msg = Message("SELL", self.name, "BARGAIN", buyer_id)
                    self.returned_msg(return_msg)
                    return return_msg
                else:
                    return_msg = Message("ASK", self.name, "BARGAIN", WTA)
                    self.returned_msg(return_msg)
                    return return_msg
            else:
                return_msg = Message("ASK", self.name, "BARGAIN", WTA)
                self.returned_msg(return_msg)
                return return_msg

    def contract(self, pl):
        """
        Update contract information for ZID Trader
        """
        contract = pl
        price = contract[1]
        buyer_id = contract[2]
        seller_id = contract[3]
        if self.type == 'BUYER':
            if self.get_name() != buyer_id:
                return_msg = Message("BAD", self.name, "BARGAIN",
                                     "08 Not buyer contract")
                self.returned_msg(return_msg)
                return return_msg
            self.units_transacted += 1
            self.cur_unit += 1
        else:  # SELLER
            if self.get_name() != seller_id:
                return_msg = Message("BAD", self.name, "BARGAIN",
                                     "09 Not seller contract")
                self.returned_msg(return_msg)
                return return_msg
            self.units_transacted += 1
            self.cur_unit += 1

        return_msg = Message("Update", self.name, "BARGAIN",
                             "10 Units Updated")
        self.returned_msg(return_msg)
        return return_msg


### Zero Intelligence-Affinity: Class ZIDA (Trader)

In [7]:
class ZIDA(Trader):
    """
        Zero Intelligence variant for decentralized market
        with Affinity to other traders
        <==> Bias to stay in current location
    """
    
    def move_requested(self, pl):
        """
        Make a move in a random direction but with bias to stay if you can still trade
        Stickiness to state quo is determined by the contract number in the last day
        """
        # get whole contract dic from class SimulateMarket
        contracts_whole = self.simulation.contracts
        # get contract dic in the last week, key = week, make it a list
        bias = 0
        if len(contracts_whole) > 0:
            contracts_week = list(contracts_whole.items())[-1][-1]
            # transfer dic to list, get the last day's contract list (second last value in the dic)
            if len(contracts_week) > 1:
                contracts_day = list(contracts_week.items())[-2][-1]
                # bias to stay = number of accumulated contract the trader make in the last day(0 in every Monday)
                for i in contracts_day:
                        if self.name == i[-1] or self.name == i[-2]:
                            bias = bias + 1
        direction_list = [-1, 0, +1] + 10 * bias * [0] #


## Test and Store results
* Create a folder `sim_name` in the catalogue
* In the folder
    * supply_demand_prices graph for every week
    * `Result_json.json` : result_whole = {}, result_whole\[week\]= {}

% ### ZID vs. ZID

In [8]:
import dmarket_shan as dm  
import os
import json

ZID = dm.ZID
ZIDA = dm.ZIDA

trader_objects = [ZID, ZID]     # List of artificial traders length 2
debug = False
num_weeks = 20                    # Number of weeks
num_periods = 7                   # 7 days a week
num_rounds = 30                   # bargaining per day
grid_size = 10                    # 10*10
num_traders = grid_size * 4       # traders (multiple of two)
num_units = 3                     # Number of units per trader
sim_name = "Decentralized-ZID"   # will be the folder name

if os.path.exists(sim_name):      # delete all files if folder exists
    for files in os.listdir(sim_name):
        file = sim_name +'/' + files
        os.remove(file)
if not os.path.exists(sim_name):  # create new folder to store result(dict)
    os.makedirs(sim_name)

sim_mkt = dm.SimulateMarket(sim_name, num_weeks, num_periods, num_rounds, num_traders, trader_objects, grid_size,
                         num_units, debug)
sim_mkt.make_whole_simulation()
    




============================ Week 0
=============== starting grid
at point (0, 1) we have 1 agents = ['S_5_ZID']
at point (0, 4) we have 1 agents = ['B_9_ZID']
at point (0, 6) we have 2 agents = ['B_10_ZID', 'B_20_ZID']
at point (1, 4) we have 1 agents = ['S_1_ZID']
at point (1, 5) we have 1 agents = ['S_2_ZID']
at point (1, 6) we have 1 agents = ['B_3_ZID']
at point (1, 7) we have 1 agents = ['B_8_ZID']
at point (1, 9) we have 1 agents = ['S_4_ZID']
at point (2, 0) we have 1 agents = ['S_14_ZID']
at point (2, 8) we have 1 agents = ['S_17_ZID']
at point (2, 9) we have 1 agents = ['S_19_ZID']
at point (3, 3) we have 2 agents = ['B_5_ZID', 'S_3_ZID']
at point (3, 5) we have 1 agents = ['S_13_ZID']
at point (4, 9) we have 1 agents = ['S_20_ZID']
at point (5, 1) we have 2 agents = ['B_1_ZID', 'S_10_ZID']
at point (5, 3) we have 1 agents = ['S_6_ZID']
at point (6, 1) we have 1 agents = ['B_13_ZID']
at point (6, 2) we have 1 agents = ['B_7_ZID']
at point (6, 3) we have 1 agents = ['B_18_ZID'

---------------------------------------- Results in Week 2
Market Decentralized-ZID has 40 traders, each with 3 units
      equilibrium price range is 301 to 301
      average price = 305.9230769230769
      equilibrium expected contracts = 29, actual contracts = 13
      buyer surplus = 639, seller surplus = 603
      actual_surplus = 1242, maximum surplus = 2494
      efficiency = 49.80%.

  ZID has surplus 1242
============================ Week 3
=============== starting grid
at point (0, 3) we have 1 agents = ['S_3_ZID']
at point (0, 6) we have 1 agents = ['S_1_ZID']
at point (0, 7) we have 1 agents = ['B_20_ZID']
at point (1, 0) we have 1 agents = ['B_13_ZID']
at point (1, 1) we have 2 agents = ['B_18_ZID', 'S_6_ZID']
at point (1, 4) we have 1 agents = ['S_13_ZID']
at point (1, 6) we have 1 agents = ['S_4_ZID']
at point (2, 2) we have 1 agents = ['B_9_ZID']
at point (2, 3) we have 1 agents = ['B_14_ZID']
at point (2, 9) we have 2 agents = ['B_8_ZID', 'S_19_ZID']
at point (3, 7) we

---------------------------------------- Results in Week 6
Market Decentralized-ZID has 40 traders, each with 3 units
      equilibrium price range is 288 to 289
      average price = 277.25
      equilibrium expected contracts = 31, actual contracts = 20
      buyer surplus = 849, seller surplus = 213
      actual_surplus = 1062, maximum surplus = 3307
      efficiency = 32.11%.

  ZID has surplus 1062
============================ Week 7
=============== starting grid
at point (0, 6) we have 1 agents = ['S_3_ZID']
at point (0, 8) we have 2 agents = ['B_3_ZID', 'B_14_ZID']
at point (1, 3) we have 1 agents = ['S_13_ZID']
at point (1, 8) we have 1 agents = ['B_20_ZID']
at point (1, 9) we have 1 agents = ['S_19_ZID']
at point (2, 7) we have 1 agents = ['S_20_ZID']
at point (3, 1) we have 1 agents = ['B_19_ZID']
at point (3, 3) we have 1 agents = ['B_18_ZID']
at point (3, 8) we have 2 agents = ['S_1_ZID', 'S_6_ZID']
at point (3, 9) we have 1 agents = ['S_5_ZID']
at point (4, 2) we have 1 ag

---------------------------------------- Results in Week 10
Market Decentralized-ZID has 40 traders, each with 3 units
      equilibrium price range is 297 to 299
      average price = 280.6666666666667
      equilibrium expected contracts = 30, actual contracts = 15
      buyer surplus = 632, seller surplus = 338
      actual_surplus = 970, maximum surplus = 2667
      efficiency = 36.37%.

  ZID has surplus 970
============================ Week 11
=============== starting grid
at point (1, 1) we have 1 agents = ['B_10_ZID']
at point (1, 5) we have 1 agents = ['B_20_ZID']
at point (1, 7) we have 1 agents = ['S_9_ZID']
at point (1, 8) we have 1 agents = ['S_5_ZID']
at point (2, 4) we have 1 agents = ['S_3_ZID']
at point (2, 6) we have 1 agents = ['B_18_ZID']
at point (2, 8) we have 1 agents = ['B_2_ZID']
at point (3, 0) we have 2 agents = ['S_7_ZID', 'S_13_ZID']
at point (3, 1) we have 1 agents = ['B_9_ZID']
at point (3, 4) we have 1 agents = ['S_8_ZID']
at point (3, 8) we have 1 agent

---------------------------------------- Results in Week 14
Market Decentralized-ZID has 40 traders, each with 3 units
      equilibrium price range is 296 to 298
      average price = 290.14285714285717
      equilibrium expected contracts = 26, actual contracts = 14
      buyer surplus = 831, seller surplus = 203
      actual_surplus = 1034, maximum surplus = 2590
      efficiency = 39.92%.

  ZID has surplus 1034
============================ Week 15
=============== starting grid
at point (0, 0) we have 1 agents = ['S_8_ZID']
at point (0, 1) we have 1 agents = ['B_10_ZID']
at point (0, 2) we have 1 agents = ['B_18_ZID']
at point (0, 7) we have 1 agents = ['B_2_ZID']
at point (0, 9) we have 1 agents = ['S_11_ZID']
at point (1, 0) we have 1 agents = ['S_17_ZID']
at point (1, 4) we have 1 agents = ['B_19_ZID']
at point (1, 5) we have 2 agents = ['B_6_ZID', 'B_16_ZID']
at point (2, 0) we have 1 agents = ['S_13_ZID']
at point (2, 2) we have 1 agents = ['S_4_ZID']
at point (2, 3) we have 1

---------------------------------------- Results in Week 17
Market Decentralized-ZID has 40 traders, each with 3 units
      equilibrium price range is 308 to 309
      average price = 305.7142857142857
      equilibrium expected contracts = 30, actual contracts = 21
      buyer surplus = 607, seller surplus = 1335
      actual_surplus = 1942, maximum surplus = 2825
      efficiency = 68.74%.

  ZID has surplus 1942
============================ Week 18
=============== starting grid
at point (0, 1) we have 1 agents = ['B_8_ZID']
at point (0, 2) we have 1 agents = ['S_19_ZID']
at point (0, 8) we have 1 agents = ['B_13_ZID']
at point (0, 9) we have 1 agents = ['S_11_ZID']
at point (1, 0) we have 2 agents = ['B_14_ZID', 'B_16_ZID']
at point (1, 5) we have 1 agents = ['S_8_ZID']
at point (1, 9) we have 1 agents = ['B_2_ZID']
at point (2, 2) we have 1 agents = ['B_18_ZID']
at point (3, 0) we have 2 agents = ['B_20_ZID', 'S_17_ZID']
at point (3, 6) we have 1 agents = ['S_7_ZID']
at point (4, 

### ZIDA vs. ZIDA

In [ ]:
import dmarket_shan as dm  
import os
import json

ZID = dm.ZID
ZIDA = dm.ZIDA

trader_objects = [ZIDA, ZIDA]     # List of artificial traders length 2
debug = False
num_weeks = 20                   # Number of weeks
num_periods = 7                   # 7 days a week
num_rounds = 30                   # bargaining per day
grid_size = 10                  # 10*10
num_traders = 100       # traders (multiple of two)
num_units =  1                    # Number of units per trader
sim_name = "Decentralized-ZIDA"   # will be the folder name

if os.path.exists(sim_name):      # delete all files if folder exists
    for files in os.listdir(sim_name):
        file = sim_name +'/' + files
        os.remove(file)
if not os.path.exists(sim_name):  # create new folder to store result(dict)
    os.makedirs(sim_name)

sim_mkt = dm.SimulateMarket(sim_name, num_weeks, num_periods, num_rounds, num_traders, trader_objects, grid_size,
                         num_units, debug)
sim_mkt.make_whole_simulation()
    


% ### ZID VS. ZIDA

In [9]:
import dmarket_shan as dm  
import os
import json

ZID = dm.ZID
ZIDA = dm.ZIDA

trader_objects = [ZID, ZIDA]     # List of artificial traders length 2
debug = False
num_weeks = 20                    # Number of weeks
num_periods = 7                   # 7 days a week
num_rounds = 30                   # bargaining per day
grid_size = 10                    # 10*10
num_traders = grid_size * 4       # traders (multiple of two)
num_units = 8                     # Number of units per trader
sim_name = "Decentralized-ZID-ZIDA"   # will be the folder name

if os.path.exists(sim_name):      # delete all files if folder exists
    for files in os.listdir(sim_name):
        file = sim_name +'/' + files
        os.remove(file)
if not os.path.exists(sim_name):  # create new folder to store result(dict)
    os.makedirs(sim_name)

sim_mkt = dm.SimulateMarket(sim_name, num_weeks, num_periods, num_rounds, num_traders, trader_objects, grid_size,
                         num_units, debug)
sim_mkt.make_whole_simulation()

============================ Week 0
=============== starting grid
at point (0, 0) we have 1 agents = ['B_13_ZID']
at point (0, 1) we have 1 agents = ['B_17_ZIDA']
at point (0, 6) we have 2 agents = ['B_12_ZID', 'S_8_ZIDA']
at point (0, 8) we have 1 agents = ['B_4_ZIDA']
at point (1, 5) we have 1 agents = ['S_17_ZID']
at point (1, 6) we have 1 agents = ['S_15_ZIDA']
at point (1, 7) we have 1 agents = ['S_5_ZIDA']
at point (2, 4) we have 1 agents = ['S_19_ZIDA']
at point (2, 5) we have 1 agents = ['B_7_ZIDA']
at point (3, 3) we have 2 agents = ['B_2_ZID', 'S_13_ZIDA']
at point (3, 6) we have 1 agents = ['B_8_ZIDA']
at point (3, 7) we have 2 agents = ['B_3_ZID', 'S_11_ZIDA']
at point (3, 9) we have 1 agents = ['B_18_ZID']
at point (4, 0) we have 1 agents = ['B_20_ZIDA']
at point (4, 3) we have 1 agents = ['B_1_ZIDA']
at point (4, 6) we have 1 agents = ['S_7_ZIDA']
at point (4, 7) we have 1 agents = ['S_16_ZIDA']
at point (5, 3) we have 1 agents = ['S_3_ZID']
at point (5, 5) we have 1 agen

/Users/gs/Library/CloudStorage/OneDrive-Personal/Notes-Jupyter/Econ895/dmarket_shan.py:733: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 7.5))  # Set plot dimensions


---------------------------------------- Results in Week 2
Market Decentralized-ZID-ZIDA has 40 traders, each with 8 units
      equilibrium price range is 302 to 302
      average price = 305.2105263157895
      equilibrium expected contracts = 79, actual contracts = 38
      buyer surplus = 1813, seller surplus = 1413
      actual_surplus = 3226, maximum surplus = 7706
      efficiency = 41.86%.

  ZIDA has surplus 1377
  ZID has surplus 1849
============================ Week 3
=============== starting grid
at point (0, 6) we have 1 agents = ['S_9_ZID']
at point (0, 7) we have 1 agents = ['S_15_ZIDA']
at point (1, 3) we have 1 agents = ['B_4_ZIDA']
at point (1, 6) we have 1 agents = ['S_5_ZIDA']
at point (1, 7) we have 1 agents = ['B_3_ZID']
at point (2, 2) we have 2 agents = ['B_7_ZIDA', 'B_13_ZID']
at point (2, 5) we have 1 agents = ['B_8_ZIDA']
at point (2, 6) we have 1 agents = ['S_19_ZIDA']
at point (3, 0) we have 1 agents = ['S_8_ZIDA']
at point (3, 2) we have 1 agents = ['S_13

CONTRACTS
---------
2 291 B_1_ZIDA S_6_ZID
4 290 B_1_ZIDA S_6_ZID
13 373 B_1_ZIDA S_6_ZID
2 349 B_13_ZID S_19_ZIDA
22 286 B_13_ZID S_20_ZID
1 254 B_9_ZID S_4_ZID
16 330 B_9_ZID S_4_ZID
6 296 B_11_ZID S_2_ZID
9 287 B_11_ZID S_2_ZID
12 339 B_11_ZID S_2_ZID
9 268 B_13_ZID S_19_ZIDA
0 371 B_12_ZID S_7_ZIDA
13 287 B_12_ZID S_7_ZIDA
14 334 B_12_ZID S_7_ZIDA
1 276 B_13_ZID S_13_ZIDA
2 227 B_13_ZID S_13_ZIDA
5 213 B_15_ZID S_16_ZIDA
9 314 B_16_ZIDA S_16_ZIDA
17 295 B_16_ZIDA S_16_ZIDA
23 341 B_16_ZIDA S_16_ZIDA
2 394 B_20_ZIDA S_12_ZIDA
7 344 B_20_ZIDA S_12_ZIDA
2 354 B_2_ZID S_3_ZID
9 230 B_2_ZID S_3_ZID
13 291 B_2_ZID S_3_ZID
15 341 B_2_ZID S_3_ZID
0 268 B_8_ZIDA S_5_ZIDA
13 249 B_8_ZIDA S_5_ZIDA
23 340 B_15_ZID S_16_ZIDA
12 256 B_16_ZIDA S_11_ZIDA
19 298 B_16_ZIDA S_11_ZIDA
26 261 B_16_ZIDA S_11_ZIDA
1 254 B_19_ZID S_15_ZIDA
2 252 B_19_ZID S_15_ZIDA
4 300 B_19_ZID S_15_ZIDA
10 290 B_19_ZID S_15_ZIDA
15 296 B_19_ZID S_15_ZIDA
29 251 B_19_ZID S_15_ZIDA
7 288 B_20_ZIDA S_12_ZIDA
22 302 B_20_ZI

---------------------------------------- Results in Week 9
Market Decentralized-ZID-ZIDA has 40 traders, each with 8 units
      equilibrium price range is 293 to 293
      average price = 297.1166666666667
      equilibrium expected contracts = 92, actual contracts = 60
      buyer surplus = 3249, seller surplus = 2378
      actual_surplus = 5627, maximum surplus = 9005
      efficiency = 62.49%.

  ZIDA has surplus 2985
  ZID has surplus 2642
============================ Week 10
=============== starting grid
at point (0, 2) we have 1 agents = ['B_10_ZID']
at point (0, 9) we have 1 agents = ['B_7_ZIDA']
at point (1, 9) we have 1 agents = ['S_15_ZIDA']
at point (2, 7) we have 1 agents = ['S_13_ZIDA']
at point (3, 3) we have 2 agents = ['B_8_ZIDA', 'S_7_ZIDA']
at point (3, 5) we have 1 agents = ['B_3_ZID']
at point (3, 8) we have 1 agents = ['B_13_ZID']
at point (3, 9) we have 2 agents = ['B_19_ZID', 'S_5_ZIDA']
at point (4, 4) we have 2 agents = ['B_20_ZIDA', 'S_8_ZIDA']
at point (4, 8

CONTRACTS
---------
5 263 B_5_ZIDA S_3_ZID
23 292 B_5_ZIDA S_3_ZID
25 313 B_5_ZIDA S_3_ZID
8 242 B_17_ZIDA S_4_ZID
9 213 B_17_ZIDA S_4_ZID
29 263 B_17_ZIDA S_4_ZID
8 255 B_20_ZIDA S_13_ZIDA
9 303 B_20_ZIDA S_13_ZIDA
10 227 B_20_ZIDA S_13_ZIDA
18 261 B_20_ZIDA S_13_ZIDA
11 355 B_6_ZIDA S_4_ZID
10 237 B_20_ZIDA S_12_ZIDA
2 248 B_1_ZIDA S_18_ZID
4 283 B_1_ZIDA S_18_ZID
7 333 B_15_ZID S_4_ZID
18 309 B_15_ZID S_4_ZID
22 353 B_6_ZIDA S_4_ZID
2 369 B_7_ZIDA S_16_ZIDA
6 277 B_7_ZIDA S_16_ZIDA
8 308 B_7_ZIDA S_16_ZIDA
14 250 B_7_ZIDA S_16_ZIDA
0 306 B_11_ZID S_19_ZIDA
4 340 B_11_ZID S_19_ZIDA
7 294 B_11_ZID S_19_ZIDA
5 327 B_14_ZIDA S_7_ZIDA
6 334 B_14_ZIDA S_7_ZIDA
0 274 B_16_ZIDA S_10_ZID
10 260 B_16_ZIDA S_10_ZID
2 299 B_14_ZIDA S_7_ZIDA
22 272 B_14_ZIDA S_7_ZIDA
19 313 B_16_ZIDA S_10_ZID
9 296 B_2_ZID S_2_ZID
10 250 B_2_ZID S_2_ZID
11 279 B_2_ZID S_2_ZID
14 333 B_15_ZID S_7_ZIDA
--------------------
B_1_ZIDA moved [(5, 1), (4, 0), (4, 0), (4, 1), (4, 2), (4, 2), (4, 2), (3, 1)]
B_2_ZID move

---------------------------------------- Results in Week 16
Market Decentralized-ZID-ZIDA has 40 traders, each with 8 units
      equilibrium price range is 294 to 294
      average price = 290.64444444444445
      equilibrium expected contracts = 80, actual contracts = 45
      buyer surplus = 2526, seller surplus = 1148
      actual_surplus = 3674, maximum surplus = 7873
      efficiency = 46.67%.

  ZIDA has surplus 1973
  ZID has surplus 1701
============================ Week 17
=============== starting grid
at point (0, 0) we have 1 agents = ['B_3_ZID']
at point (0, 7) we have 1 agents = ['B_7_ZIDA']
at point (0, 9) we have 1 agents = ['B_2_ZID']
at point (1, 1) we have 2 agents = ['B_1_ZIDA', 'B_15_ZID']
at point (1, 3) we have 1 agents = ['S_2_ZID']
at point (1, 4) we have 1 agents = ['B_10_ZID']
at point (1, 9) we have 1 agents = ['S_17_ZID']
at point (2, 1) we have 1 agents = ['S_1_ZIDA']
at point (2, 4) we have 1 agents = ['B_5_ZIDA']
at point (3, 2) we have 1 agents = ['S_6_

## Playback: PyGame
    Assume a grid of 10 x 10.
    Build a PyGame program to playback your simulation.

Better to run pygame in terminal

    python pygame_playback_shan.py
    * change sim_name for different Strategy

% ### ZID vs. ZID

In [ ]:
import json
import pygame
import random
import time

# import some event_constants
from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    K_s,
    K_p,
    KEYDOWN,
    QUIT,
)

# Load Results dictionary
sim_name="Decentralized-ZID"
Results = json.load(open(sim_name + "/Results_json.json", "r"))

# Sep up color schemes
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

# Define constants for simulation
DIM = Results["0"]["grid_size"]                 # edge of square grid
NUM_AGENTS = Results["0"]["number traders"]
FPS = 1                                         # frames per second (lower is slower)
STICK = 5                                       # stickiness of location 4**num_of_agents
BIAS = 2                                        # down-right bias
DIM_edge = 60                                   # edge for trader's location square

# Define constants for screen
SCREEN_WIDTH = DIM * DIM_edge + 800
SCREEN_HEIGHT = DIM * DIM_edge + 200




def make_grid(agents):
    """
        Returns grid dictionary such that,
        grid[loc] = name (of agent),
                    where loc = (x, y) coordinates.
    """
    grid = {}
    for agent in agents:
        x_loc, y_loc = agents[agent]['locs']
        name = agents[agent]['name']
        if (x_loc, y_loc) in grid.keys():
            grid[(x_loc, y_loc)].append(name)
        else:
            grid[(x_loc, y_loc)] = [name]
    return grid


def make_agents():
    """Return dictionary with num_agents"""
    agents = {}
    k = 0
    for name in Results["0"]["Moving_History"]:
        agents[k] = {}
        x = Results["0"]["Moving_History"][name][0][0]
        y = Results["0"]["Moving_History"][name][0][1]
        agents[k]['locs'] = (x, y)        #get the initial location
        agents[k]['name'] = name
        agents[k]['meet'] = False
        k = k + 1
    return agents


def agent_lookup(agents, name):
    for agent in agents:
        if agents[agent]['name'] == name:
            return agent
    print("Bad Mojo")


def move_agents(agents, week, day):
    """Move agents in random unit directions"""
    k = 0
    for name in Results["0"]["Moving_History"]:
        x = Results[week]["Moving_History"][name][day][0]
        y = Results[week]["Moving_History"][name][day][1]
        agents[k]['locs'] = (x, y)
        k = k + 1
    return agents


agents = make_agents()

time_now = 0
fps = FPS
start = False
pygame.init()
font = pygame.font.SysFont('Arial', 20)
clock = pygame.time.Clock()

# Set up the drawing window
pygame.display.set_caption('Payback - ' + Results["0"]["name"])
screen = pygame.display.set_mode([SCREEN_WIDTH, SCREEN_HEIGHT])

# Run until the user asks to quit
running = True
while running:
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False
            elif event.key == K_s:
                start = True
            elif event.key == K_p:
                start = False
            elif event.key == K_UP:
                fps = fps + 1
                if fps >= 50: fps = 50
            elif event.key == K_DOWN:
                fps = fps - 1
                if fps <= 1: fps = 1
        elif event.type == QUIT:
            running = False

    rect = pygame.Rect(5, 10, 50, 100)
    screen_info = f"  Moving History               FPS: {fps}"
    screen_info_1 = "  (Buyer = Green, Seller = Blue, Meet = Red)"
    screen_instruct = "  s-start, p-pause, exc-quit, up-fps, down-fps"


    if not start:
        pygame.draw.rect(screen, WHITE, (5, 5, SCREEN_WIDTH, 100))
        screen.blit(font.render(screen_info, True, BLACK), (5, 5))
        screen.blit(font.render(screen_info_1, True, BLACK), (5, 25))
        screen.blit(font.render(screen_instruct, True, BLACK), (5, 50))

    if start:
        # remove previews content in the surface
        screen.fill(WHITE)
        

        for week in Results.keys():
            # load graph with supply, demand, prices in this week
            pic_name = str(sim_name) + "/Week"+str(week)+"supply_demand.jpg"
            Img = pygame.image.load(pic_name)
            # adjust picture size
            Img = pygame.transform.scale(Img, (500, 300))
            screen.blit(Img, (100 + (DIM+2) * DIM_edge, 100))
            
            # get needed information in this week from Results
            result = Results[week]
            trade = f"This is Week {week}, number of traders =  {result['number traders']} , each can trade {result['number units']} units"
            prices = f"Equilibrium price is from {result['eq low price']}  to  {result['eq high price']}  "
            contract = f"Equilibrium quantity =  {result['eq quantity']} ; Real quantity of contract = {result['number traders']}"
            surplus = f"Actual total surplus =  {result['actual surplus']} ; Buyer surplus = {result['buyer surplus']} ; Seller surplus = {result['seller surplus']} "
            efficiency = f"Efficiency (%) =  {result['efficiency']}"
            
            # set up for each line
            screen.fill(WHITE, (100 + (DIM + 2) * DIM_edge, 450, SCREEN_HEIGHT, SCREEN_WIDTH))
            screen.blit(font.render(trade, True, BLACK), (100 + (DIM + 2) * DIM_edge, 450))
            screen.blit(font.render(prices, True, BLACK), (100 + (DIM + 2) * DIM_edge, 475))
            screen.blit(font.render(contract , True, BLACK), (100 + (DIM + 2) * DIM_edge, 500))
            screen.blit(font.render(surplus, True, BLACK), (100 + (DIM + 2) * DIM_edge, 525))
            screen.blit(font.render(efficiency, True, BLACK), (100 + (DIM + 2) * DIM_edge, 550))

            for day in range(Results[week]["num_days"]):
                screen.fill(WHITE, (0, 0, SCREEN_HEIGHT, SCREEN_HEIGHT))
                # Display Grid
                for x in range(100, 100 + (DIM + 1) * DIM_edge, DIM_edge):
                    # for x in range(100, 400, 40):
                    for y in range(100, 100 + (DIM + 1) * DIM_edge, DIM_edge):
                        pygame.draw.circle(screen, BLACK, (x, y), 2)

                # add one row to show current week and current day
                period_ls = ["New Start: Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
                screen_week_day = f"  Week: {week}    {period_ls[day]}"
                time.sleep(0.8)
                agents = move_agents(agents, week, day)
                grid = make_grid(agents)
                pygame.display.flip()

                num_b = 0
                num_s = 0
                # Display Agents
                for loc in grid:
                    x_loc, y_loc = loc
                    x_b = x_loc * DIM_edge + 100
                    y_b = y_loc * DIM_edge + 100
                    
                    # seller will be shown in the a little to the right in that grid
                    x_s = x_loc * DIM_edge + 100 + DIM_edge / 4
                    y_s = y_loc * DIM_edge + 100

                    print_color_s = GREEN
                    print_color_b = BLUE
                    # Check Types at location
                    buyer = False
                    seller = False
                    num_b = 0
                    num_s = 0
                    for name in grid[loc]:
                        if name[:1] == 'B':
                            num_b = num_b + 1
                            print_color_s = RED
                            screen.blit(font.render(name, True, BLACK), (x_b, y_b + num_b * DIM_edge / 2))
                            rect_b = pygame.Rect(x_b, y_b, DIM_edge / 4, DIM_edge / 4 * num_b)
                        else:
                            num_s = num_s + 1
                            print_color_b = RED
                            screen.blit(font.render(name, True, BLACK), (x_s, y_s + num_s * DIM_edge / 2))
                            rect_s = pygame.Rect(x_s, y_s, DIM_edge / 4, DIM_edge / 4 * num_s)
                        if num_b > 0:
                            pygame.draw.rect(screen, print_color_b, rect_b)
                            pygame.display.flip()
                        if num_s > 0:
                            pygame.draw.rect(screen, print_color_s, rect_s)
                            pygame.display.flip()

                    screen.blit(font.render(screen_info, True, BLACK), (5, 5))
                    screen.blit(font.render(screen_info_1, True, BLACK), (5, 25))
                    screen.blit(font.render(screen_instruct, True, BLACK), (5, 50))
                    screen.blit(font.render(screen_week_day, True, BLACK), (5, 75))

                    time_now = time_now + 1


    pygame.display.flip()
    clock.tick(fps)
pygame.quit()



pygame 2.5.2 (SDL 2.28.3, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


### ZIDA vs. ZIDA

In [9]:
import json
import pygame
import os
import numpy as np
import cv2

from pygame.locals import (
    K_ESCAPE, K_s, K_p, K_UP, K_DOWN, K_r,
    KEYDOWN, QUIT
)

# ============================================================
# Load Results
# ============================================================
sim_name = "Decentralized-ZIDA"
Results = json.load(open(sim_name + "/Results_json.json", "r"))

DIM = Results["0"]["grid_size"]

# ============================================================
# 16:9 Page Layout (fixed)
#   - Left half: grid
#   - Right half: figure + summary
# ============================================================
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 720

MARGIN = 18
TOPBAR_H = 78

LEFT_W = SCREEN_WIDTH // 2
RIGHT_W = SCREEN_WIDTH - LEFT_W

LEFT_X0 = 0
RIGHT_X0 = LEFT_W

CONTENT_Y0 = TOPBAR_H
CONTENT_H = SCREEN_HEIGHT - TOPBAR_H

# grid cell size chosen to fit within left half content area
MAX_GRID_W = LEFT_W - 2 * MARGIN
MAX_GRID_H = CONTENT_H - 2 * MARGIN
CELL = max(18, int(min(MAX_GRID_W / DIM, MAX_GRID_H / DIM)))  # keep readable minimum

GRID_W = DIM * CELL
GRID_H = DIM * CELL

GRID_ORIGIN = (
    LEFT_X0 + (LEFT_W - GRID_W) // 2,
    CONTENT_Y0 + (CONTENT_H - GRID_H) // 2
)

# right panel content box
RIGHT_PAD = 18
RIGHT_BOX = (
    RIGHT_X0 + RIGHT_PAD,
    CONTENT_Y0 + RIGHT_PAD,
    RIGHT_W - 2 * RIGHT_PAD,
    CONTENT_H - 2 * RIGHT_PAD
)

# ============================================================
# Playback speed (decoupled)
# ============================================================
RENDER_FPS_DEFAULT = 30     # smooth rendering
DAY_SECONDS_DEFAULT = 1.2   # slower initial playback: 1 day per 1.2 seconds

# ============================================================
# Palette
# ============================================================
BG = (250, 250, 252)
INK = (25, 28, 35)
MUTED = (120, 130, 145)
GRIDLINE = (225, 230, 238)

BUYER = (46, 160, 67)       # green-ish
SELLER = (45, 114, 210)     # blue-ish
BOTH = (220, 75, 57)        # red-ish

PANEL_BG = (255, 255, 255)
PANEL_BORDER = (230, 235, 242)

# ============================================================
# Coordinate normalization (fix 1-based vs 0-based)
# ============================================================
def detect_and_get_coord_shift(results, dim):
    xs, ys = [], []
    mh = results["0"]["Moving_History"]
    for name in mh:
        x, y = mh[name][0]
        xs.append(x)
        ys.append(y)

    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)

    looks_one_based = (min_x >= 1 and min_y >= 1 and max_x <= dim and max_y <= dim)
    if looks_one_based and (max_x == dim or max_y == dim):
        return 1
    return 0

COORD_SHIFT = detect_and_get_coord_shift(Results, DIM)

def normalize_loc(x, y):
    if COORD_SHIFT == 1:
        x -= 1
        y -= 1
    return x, y

# ============================================================
# Helpers (your logic)
# ============================================================
def make_agents():
    agents = {}
    k = 0
    for name in Results["0"]["Moving_History"]:
        x = Results["0"]["Moving_History"][name][0][0]
        y = Results["0"]["Moving_History"][name][0][1]
        x, y = normalize_loc(x, y)
        agents[k] = {"locs": (x, y), "name": name}
        k += 1
    return agents

def move_agents(agents, week, day):
    k = 0
    for name in Results["0"]["Moving_History"]:
        x = Results[str(week)]["Moving_History"][name][day][0]
        y = Results[str(week)]["Moving_History"][name][day][1]
        x, y = normalize_loc(x, y)
        agents[k]["locs"] = (x, y)
        k += 1
    return agents

def make_grid(agents):
    grid = {}
    for _, a in agents.items():
        loc = a["locs"]
        grid.setdefault(loc, []).append(a["name"])
    return grid

def is_buyer(name: str) -> bool:
    return name[:1].upper() == "B"

def fmt2(x):
    """Format numeric-like to 2 decimals; fallback to str if not parseable."""
    try:
        return f"{float(x):.2f}"
    except Exception:
        return str(x)

# ============================================================
# Pygame setup
# ============================================================
pygame.init()
pygame.display.set_caption("Playback - " + Results["0"]["name"])
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()

font_title = pygame.font.SysFont("Arial", 24, bold=True)
font = pygame.font.SysFont("Arial", 16)
font_small = pygame.font.SysFont("Arial", 14)

# Cache week images once (store original; scale dynamically each frame)
week_imgs = {}
for wk in Results.keys():
    path = f"{sim_name}/Week{wk}supply_demand.jpg"
    if os.path.exists(path):
        week_imgs[str(wk)] = pygame.image.load(path).convert()

# ============================================================
# Video recording state
# ============================================================
recording = False
video = None
VIDEO_FILENAME = "playback.mp4"

def start_recording(filename=VIDEO_FILENAME, fps=30):
    # Overwrite existing video if present
    if os.path.exists(filename):
        try:
            os.remove(filename)
        except OSError:
            pass

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # widely playable MP4
    vw = cv2.VideoWriter(filename, fourcc, fps, (SCREEN_WIDTH, SCREEN_HEIGHT))
    if not vw.isOpened():
        raise RuntimeError(
            "VideoWriter failed to open. If playback.mp4 is open in another program, close it and try again."
        )
    return vw

def write_frame_to_video(vw):
    frame = pygame.surfarray.array3d(screen)   # (W,H,3)
    frame = np.transpose(frame, (1, 0, 2))     # (H,W,3)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    vw.write(frame)

# ============================================================
# Drawing
# ============================================================
def draw_topbar(render_fps, day_seconds, start, week, day_label):
    pygame.draw.rect(screen, PANEL_BG, (0, 0, SCREEN_WIDTH, TOPBAR_H))
    pygame.draw.line(screen, PANEL_BORDER, (0, TOPBAR_H), (SCREEN_WIDTH, TOPBAR_H), 1)

    title = "Decentralized ZI Affinity Trader — Playback"
    screen.blit(font_title.render(title, True, INK), (MARGIN, 16))

    status = "RUNNING" if start else "PAUSED"
    rec = "REC ●" if recording else "REC"
    coord_note = "1-based→0-based" if COORD_SHIFT == 1 else "0-based"
    info = (
        f"[S] start  [P] pause  [↑/↓] speed  [R] record on/off  [ESC] quit   |   "
        f"{status}   |   RenderFPS={render_fps}   |   DayStep={day_seconds:.1f}s   |   "
        f"Week {week}  {day_label}   |   {rec}   |   Coord={coord_note}"
    )
    screen.blit(font.render(info, True, MUTED), (MARGIN, 46))

def draw_left_panel_background():
    # subtle panel background for left half
    pygame.draw.rect(screen, BG, (0, TOPBAR_H, LEFT_W, SCREEN_HEIGHT - TOPBAR_H))

def draw_grid():
    x0, y0 = GRID_ORIGIN

    # outer rounded card
    card_x = x0 - 10
    card_y = y0 - 10
    card_w = GRID_W + 20
    card_h = GRID_H + 20
    pygame.draw.rect(screen, PANEL_BG, (card_x, card_y, card_w, card_h), border_radius=14)
    pygame.draw.rect(screen, PANEL_BORDER, (card_x, card_y, card_w, card_h), width=1, border_radius=14)

    # grid lines
    for i in range(DIM + 1):
        x = x0 + i * CELL
        y = y0 + i * CELL
        pygame.draw.line(screen, GRIDLINE, (x, y0), (x, y0 + GRID_H), 1)
        pygame.draw.line(screen, GRIDLINE, (x0, y), (x0 + GRID_W, y), 1)

def draw_agents(grid):
    x0, y0 = GRID_ORIGIN

    for (gx, gy), names in grid.items():
        if not (0 <= gx < DIM and 0 <= gy < DIM):
            continue

        cx = x0 + gx * CELL + CELL // 2
        cy = y0 + gy * CELL + CELL // 2

        buyers = [n for n in names if is_buyer(n)]
        sellers = [n for n in names if not is_buyer(n)]

        if buyers and sellers:
            color = BOTH
        elif buyers:
            color = BUYER
        else:
            color = SELLER

        k = len(names)
        if k == 1:
            offsets = [(0, 0)]
        else:
            # scaled offsets relative to CELL so it works for different cell sizes
            d = max(6, CELL // 6)
            base = [(-d, -d), (d, -d), (-d, d), (d, d), (0, -2*d), (0, 2*d)]
            offsets = base[:k]

        radius = max(8, CELL // 4)

        for idx, name in enumerate(names):
            ox, oy = offsets[idx]
            px, py = cx + ox, cy + oy

            pygame.draw.circle(screen, color, (px, py), radius)
            pygame.draw.circle(screen, (255, 255, 255), (px, py), radius, 2)

            label = name[:3]
            txt = font_small.render(label, True, INK)
            screen.blit(txt, (px - txt.get_width() // 2, py - txt.get_height() // 2))

def draw_right_panel(week):
    rx, ry, rw, rh = RIGHT_BOX

    pygame.draw.rect(screen, PANEL_BG, (rx, ry, rw, rh), border_radius=14)
    pygame.draw.rect(screen, PANEL_BORDER, (rx, ry, rw, rh), width=1, border_radius=14)

    result = Results[str(week)]

    # Two decimals for efficiency (and you can keep others as-is)
    eff_2d = fmt2(result.get("efficiency", ""))
    avg_2d = fmt2(result.get("avg price", ""))

    lines = [
        f"Traders: {result['number traders']}    Units/trader: {result['number units']}",
        f"Eq. price range: {result['eq low price']}  to  {result['eq high price']}",
        f"Avg price: {avg_2d}",
        f"Eq quantity: {result['eq quantity']}    Actual quantity: {result['quantity']}",
        f"Buyer surplus: {result['buyer surplus']}    Seller surplus: {result['seller surplus']}",
        f"Actual surplus: {result['actual surplus']}    Efficiency: {eff_2d}%",
    ]

    pad = 16
    line_h = 24
    stats_h = len(lines) * line_h
    stats_top = ry + rh - pad - stats_h

    # Chart box above the stats
    chart_top = ry + pad
    chart_max_h = max(120, stats_top - chart_top - 10)
    chart_max_w = rw - 2 * pad

    wk = str(week)
    if wk in week_imgs:
        src = week_imgs[wk]
        sw, sh = src.get_size()
        scale = min(chart_max_w / sw, chart_max_h / sh)
        new_size = (max(1, int(sw * scale)), max(1, int(sh * scale)))
        img = pygame.transform.smoothscale(src, new_size)
        screen.blit(img, (rx + pad, chart_top))
    else:
        screen.blit(font.render("No supply/demand image found.", True, MUTED), (rx + pad, chart_top))

    # Stats at bottom
    ty = stats_top
    for ln in lines:
        screen.blit(font.render(ln, True, INK), (rx + pad, ty))
        ty += line_h

    # Small legend (optional but useful in class)
    leg_y = stats_top - 28
    if leg_y > chart_top + 10:
        screen.blit(font_small.render("Legend:", True, MUTED), (rx + pad, leg_y))
        lx = rx + pad + 64
        items = [("Buyer", BUYER), ("Seller", SELLER), ("Both", BOTH)]
        for label, col in items:
            pygame.draw.circle(screen, col, (lx, leg_y + 8), 6)
            screen.blit(font_small.render(label, True, MUTED), (lx + 12, leg_y))
            lx += 110

# ============================================================
# Main loop state
# ============================================================
agents = make_agents()

start = False
render_fps = RENDER_FPS_DEFAULT

day_seconds = DAY_SECONDS_DEFAULT
day_timer = 0.0

week_list = sorted([int(k) for k in Results.keys()])
week_idx = 0
day = 0
period_ls = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

running = True
while running:
    dt = clock.get_time() / 1000.0

    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False
            elif event.key == K_s:
                start = True
            elif event.key == K_p:
                start = False
            elif event.key == K_UP:
                day_seconds = max(0.1, day_seconds - 0.1)   # faster
            elif event.key == K_DOWN:
                day_seconds = min(10.0, day_seconds + 0.1)  # slower
            elif event.key == K_r:
                # toggle recording; overwrites playback.mp4 if it exists
                if not recording:
                    video = start_recording(VIDEO_FILENAME, fps=30)
                    recording = True
                else:
                    recording = False
                    if video is not None:
                        video.release()
                        video = None
        elif event.type == QUIT:
            running = False

    # Update simulation clock if running
    if start:
        day_timer += dt
        if day_timer >= day_seconds:
            day_timer = 0.0

            week = week_list[week_idx]
            num_days = Results[str(week)]["num_days"]

            agents = move_agents(agents, week, day)
            day += 1

            if day >= num_days:
                day = 0
                week_idx += 1
                if week_idx >= len(week_list):
                    start = False
                    week_idx = len(week_list) - 1
                    day = num_days - 1

    # Render
    screen.fill(BG)

    current_week = week_list[week_idx]
    day_label = period_ls[day] if day < len(period_ls) else f"Day {day}"

    draw_topbar(render_fps, day_seconds, start, current_week, day_label)
    draw_left_panel_background()
    draw_grid()
    draw_agents(make_grid(agents))
    draw_right_panel(current_week)

    pygame.display.flip()

    # Save frame after drawing
    if recording and video is not None:
        write_frame_to_video(video)

    clock.tick(render_fps)

# Clean up
if video is not None:
    video.release()
pygame.quit()